In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%cd ../../stimuli_generation
import sg_utils as utils_stimuli_generation
import mis_utils
%cd ../tools/data_analysis

In [ ]:
from matplotlib import rcParams

rcParams["font.family"] = "sans-serif"
rcParams["font.sans-serif"] = ["DejaVu Sans"]

# output text as text and not paths
rcParams["svg.fonttype"] = "none"
rcParams["pdf.fonttype"] = "truetype"

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
import scipy.stats

from typing import Literal, Union, Tuple, List, Optional

import sklearn.tree
import sklearn.preprocessing
import sklearn
import sklearn.linear_model
import sklearn.svm
import sklearn.ensemble
import xgboost
import pickle
import scipy
import scipy.special
import os

from tqdm.notebook import tqdm

In [ ]:
def safe_mean(df):
    """Compute a mean over a (grouped) dataset while gracefully treating non-numeric columns."""
    new_df = {}
    for col in df.columns:
        if df[col].dtype == object:
            try:
                unique_values = df[col].unique()

                if len(unique_values) == 1:
                    value = unique_values[0]
                else:
                    value = np.nan
            except:
                value = np.nan
        else:
            value = df[col].mean()
        new_df[col] = value
        
    return pd.Series(new_df)

In [ ]:
main_df = None
lowerq_df = None
for similarity_metric, sign in zip(("dreamsim", "lpips", "dists"), (1, -1, -1)):
    main_fn = f"data/image_similarities/responses_main_{similarity_metric}.pd.pkl"
    lowerq_fn = f"data/image_similarities/responses_lower_quality_{similarity_metric}.pd.pkl"
    key = f"{similarity_metric}_similarity"
    if os.path.exists(main_fn):
        additional_main_df = pd.read_pickle(main_fn)

        if main_df is None:
            main_df = additional_main_df
        else:
            main_df[key] = sign * additional_main_df[key]
    else:
        print(f"Skipping main data for {similarity_metric} similarity, file {main_fn} not found.")

    if os.path.exists(lowerq_fn):
        additional_lowerq_df = pd.read_pickle(lowerq_fn)

        if lowerq_df is None:
            lowerq_df = additional_lowerq_df
        else:
            lowerq_df[key] = sign * additional_lowerq_df[key]
    else:
        print(f"Skipping lower quality data for {similarity_metric} similarity, file {lowerq_fn} not found.")

main_df = main_df.dropna(subset=["is_demo", "catch_trial"])
lowerq_df = lowerq_df.dropna(subset=["is_demo", "catch_trial"])

In [ ]:
main_df = pd.read_pickle("data/image_similarities/responses_main_dreamsim.pd.pkl")
lowerq_df = pd.read_pickle("data/image_similarities/responses_lower_quality_dreamsim.pd.pkl")

main_df = main_df.dropna(subset=["is_demo", "catch_trial"])
lowerq_df = lowerq_df.dropna(subset=["is_demo", "catch_trial"])

In [ ]:
print(len(main_df), len(lowerq_df))
main_df_mask = main_df["dreamsim_similarity"].isnull()
lowerq_df_mask = lowerq_df["dreamsim_similarity"].isnull()
print(f"Removing {main_df_mask.sum()} rows from main_df and {lowerq_df_mask.sum()} rows from lowerq_df.")
main_df = main_df[~main_df_mask].copy()
lowerq_df = lowerq_df[~lowerq_df_mask].copy()
print(len(main_df), len(lowerq_df))

In [ ]:
def get_train_test_data(
        main_df: pd.DataFrame,
        mode: str = "natural",
        train_test_split: Union[Literal["model"], Literal["global"]] = "model",
        model_name: str = "googlenet",
        train_test_ratio: float = 1.0,
        shuffle_features: bool = False
    ) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:

    main_subset_df = main_df[main_df["mode"] == mode].copy()

    main_subset_df = main_subset_df.rename(columns={"correct": "human_correct"})

    def add_features_and_labels(df: pd.DataFrame):
        # Duplicate every row so that we can add the flipped images/labels.
        df = df.loc[df.index.repeat(2)].reset_index(drop=True)

        labels = []
        features = []

        if len(df) > 0:
            raw_features = np.stack(df["dreamsim_similarity"].apply(lambda x: (x[:9, 9], x[:9, -1], x[10:-1, 9], x[10:-1, -1])))
    
            for i in range(0, len(raw_features), 2):
                if shuffle_features:
                    idx_1 = np.random.permutation(range(0, 8)).tolist() + [8]
                    idx_2 = np.random.permutation(range(0, 8)).tolist() + [8]
                else:
                    idx_1 = idx_2 = list(range(raw_features.shape[-1]))

                features.append(np.concatenate((raw_features[i][0][idx_1], raw_features[i][1][idx_1], raw_features[i][2][idx_2], raw_features[i][3][idx_2])))
                features.append(np.concatenate((raw_features[i][1][idx_1], raw_features[i][0][idx_1], raw_features[i][3][idx_2], raw_features[i][2][idx_2])))
                labels.append(1)
                labels.append(0)
                
        features = np.array(features)
        labels = np.array(labels)

        df["features"] = pd.Series(features.tolist(), dtype=np.dtype("object"))
        df["label"] = pd.Series(labels)

        df["features"] = df["features"].apply(lambda x: np.array(x))
        
        return df

    df_train_mask = None
    df_test_mask = None

    if train_test_split == "model":
        df_train_mask = main_subset_df["model"] == model_name
        n = int(train_test_ratio * sum(df_train_mask))
        for i in range(len(df_train_mask)):
            if df_train_mask.iloc[i]:
                n -= 1
            if n < 0:
                df_train_mask.iloc[i:] = False
                break
    else:
        shuffled_idxs = np.random.permutation(len(main_subset_df))
        df_train_mask = np.zeros(len(main_subset_df), dtype=bool)
        n_train = int(train_test_ratio * len(main_subset_df))
        train_idxs = shuffled_idxs[:n_train]
        df_train_mask[train_idxs] = True

    df_test_mask = ~df_train_mask

    train_df = main_subset_df[df_train_mask]
    test_df = main_subset_df[df_test_mask]

    train_df = add_features_and_labels(train_df)
    test_df = add_features_and_labels(test_df)

    full_df = pd.concat((train_df, test_df), ignore_index=True)

    return train_df, test_df, full_df












mode = "natural"

subset_main_df = main_df[main_df["mode"] == mode].copy()
# Filter out rows whose "model" column contains the word "hard"
subset_main_df = subset_main_df[~subset_main_df["model"].str.contains("hard")]

x = np.stack(subset_main_df["agg_dreamsim"])
x = x.reshape(len(x), -1)
y = np.array(subset_main_df["correct"])

shuffled_idxs = np.random.permutation(len(x))
x = x[shuffled_idxs]
y = y[shuffled_idxs]

y = y.reshape(-1, 1)

n_train = int(0.8 * len(x))

x_train = x[:n_train]
y_train = y[:n_train]
x_test = x[n_train:]
y_test = y[n_train:]

In [ ]:
agg_fn = np.mean
#main_df["agg_lpips"] = main_df["lpips"].apply(lambda x: (agg_fn(x[:9, 9], 0), agg_fn(x[:9, -1], 0), agg_fn(x[10:-1, 9], 0), agg_fn(x[10:-1, -1], 0)))
main_df["agg_dreamsim"] = main_df["dreamsim_similarity"].apply(lambda x: (x[:9, 9], x[:9, -1], x[10:-1, 9], x[10:-1, -1]))

In [ ]:
mode = "natural"

subset_main_df = main_df[main_df["mode"] == mode].copy()
# Filter out rows whose "model" column contains the word "hard"
subset_main_df = subset_main_df[~subset_main_df["model"].str.contains("hard")]

x = np.stack(subset_main_df["agg_dreamsim"])
x = x.reshape(len(x), -1)
y = np.array(subset_main_df["correct"])

shuffled_idxs = np.random.permutation(len(x))
x = x[shuffled_idxs]
y = y[shuffled_idxs]

y = y.reshape(-1, 1)

n_train = int(0.8 * len(x))

x_train = x[:n_train]
y_train = y[:n_train]
x_test = x[n_train:]
y_test = y[n_train:]

print(x_train.shape, x_test.shape)

In [ ]:
unique_y_levels = np.unique(y)
y_c = np.array([i for yi in y for i, zi in enumerate(unique_y_levels) if zi == yi])
assert len(y_c) == len(y)
y_train_c = np.array([i for yi in y_train for i, zi in enumerate(unique_y_levels) if zi == yi])
assert len(y_train_c) == len(y_train)
y_test_c = np.array([i for yi in y_test for i, zi in enumerate(unique_y_levels) if zi == yi])
assert len(y_test_c) == len(y_test)


In [ ]:
def fit_eval_regression_model(model, x_train, y_train, x_test, y_test):
    preprocessing_x = sklearn.preprocessing.StandardScaler()
    preprocessing_y = sklearn.preprocessing.StandardScaler()
    x_train_tf = preprocessing_x.fit_transform(x_train)
    y_train_tf = preprocessing_y.fit_transform(y_train)

    x_test_tf = preprocessing_x.transform(x_test)

    model = model.fit(x_train_tf, y_train_tf)
    y_train_pred_tf = model.predict(x_train_tf)
    y_test_pred_tf = model.predict(x_test_tf)

    if y_train_pred_tf.ndim == y_train.ndim - 1:
        y_train_pred_tf = y_train_pred_tf[..., None]
    if y_test_pred_tf.ndim == y_test.ndim - 1:
        y_test_pred_tf = y_test_pred_tf[..., None]

    y_train_pred = preprocessing_y.inverse_transform(y_train_pred_tf)
    y_test_pred = preprocessing_y.inverse_transform(y_test_pred_tf)

    train_error = np.abs(y_train_pred - y_train).mean()
    test_error = np.abs(y_test_pred - y_test).mean()
    print("Train error:", train_error, "Test error:", test_error)

    pearson_test_correlation, pearson_test_significance = scipy.stats.pearsonr(y_test_pred[..., 0], y_test[..., 0])
    pearson_train_correlation, pearson_train_significance = scipy.stats.pearsonr(y_train_pred[..., 0], y_train[..., 0])
    spearman_test_correlation, spearman_test_significance = scipy.stats.spearmanr(y_test_pred[..., 0], y_test[..., 0])
    spearman_train_correlation, spearman_train_significance = scipy.stats.spearmanr(y_train_pred[..., 0], y_train[..., 0])

    # Compute R2 score
    r2_train = sklearn.metrics.r2_score(y_train, y_train_pred)
    r2_test = sklearn.metrics.r2_score(y_test, y_test_pred)

    return (
        (y_train_pred, y_test_pred),
        (pearson_train_correlation, pearson_train_significance, spearman_train_correlation, spearman_train_significance, r2_train),
        (pearson_test_correlation, pearson_test_significance, spearman_test_correlation, spearman_test_significance, r2_test)
    )

#model = sklearn.linear_model.LinearRegression()
model = sklearn.svm.SVR(C=0.1)
#model = sklearn.tree.DecisionTreeRegressor()
#model = sklearn.ensemble.GradientBoostingRegressor()
#model = xgboost.XGBRegressor()

(
    (y_train_pred, y_test_pred),
    (pearson_train_correlation, pearson_train_significance, spearman_train_correlation, spearman_train_significance, r2_train),
    (pearson_test_correlation, pearson_test_significance, spearman_test_correlation, spearman_test_significance, r2_test)
) = fit_eval_regression_model(model, x_train, y_train, x_test, y_test)

In [ ]:
unique_y_levels

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5), sharey=True)

train_error = np.abs(y_train_pred - y_train).mean()
test_error = np.abs(y_test_pred - y_test).mean()
print("Train error:", train_error, "Test error:", test_error)

axs[0].scatter(y_train, y_train_pred, s=1, c="C0")
axs[1].scatter(y_test, y_test_pred, s=1, c="C1")
axs[0].set_title("$\\rho_p = {:.2f}, p_p = {:.2f}, \\rho_s = {:.2f}, p_s = {:.2f}, R^2 = {:.2f}$".format(pearson_train_correlation, pearson_train_significance, spearman_train_correlation, spearman_train_significance, r2_train))
axs[1].set_title("$\\rho_p = {:.2f}, p_p = {:.2f}, \\rho_s = {:.2f}, p_s = {:.2f}, R^2 = {:.2f}$".format(pearson_test_correlation, pearson_test_significance, spearman_test_correlation, spearman_test_significance, r2_test))
for ax in axs:
    ax.set_xlabel("Measured Interpretability Score")
    ax.set_ylabel("Predicted Interpretability Score")
axs[1].legend([axs[1].scatter([], [], c="C0"), axs[1].scatter([], [], c="C1")], ["Train", "Test"])

# Get Ceiling Performance

In [ ]:
def get_model_interpretability_scores(predict_fn, get_probs_fn, df):
    y_test_pred = predict_fn(np.stack(df["features"].values, 0))
    y_test_probs = get_probs_fn(np.stack(df["features"].values, 0), df["label"])
    df = pd.concat(
        (
            df[["model", "channel", "layer", "human_correct", "label"]].reset_index(drop=True),
            pd.DataFrame({"machine_confidence": y_test_probs}),
            pd.DataFrame({"prediction": y_test_pred})
        ),
        axis=1
    )
    df["machine_correct"] = df["label"] == df["prediction"]
    # Compute Interpretability Score per Unit
    df = df.groupby(["model", "channel", "layer"]).mean()

    return df

In [ ]:
def plot_raw_human_machine_accuracy_data(df, key="machine_correct", ax=None):
    ax_was_none = False
    if ax is None:
        ax_was_none = True
        _, ax = plt.subplots(1, 1)

    ax.scatter(df[key], df["human_correct"], s=1, c="C0", label=f"$\\rho_s = {scipy.stats.spearmanr(df[key], df['human_correct'])[0]:.2f}$ \n"
            f"$\\rho_p = {scipy.stats.pearsonr(df[key], df['human_correct'])[0]:.2f}$", clip_on=False)

    print("Correlations:", scipy.stats.spearmanr(df[key], df['human_correct'])[0], scipy.stats.pearsonr(df[key], df['human_correct'])[0])

    ax.set_xlabel("Machine Interpretability Score (MIS)")
    ax.set_ylabel("Human Interpretability Score (HIS)")
    ax.legend(frameon=False)
    ax.spines[["top", "right"]].set_visible(False)

    if ax_was_none:
        plt.show()

def plot_binned_human_machine_accuracy_data(df, key="machine_correct", ax=None, bins="unique", compact_label=False):
    """Plot the average human and machine accuracy for each bin of machine accuracy.
    As the machine accuracy can only take discrete values, we will use these values to bin the data.
    We plot the mean and standard deviation of the human accuracy for each bin."""
    
    if bins == "unique":
        mean_df = safe_mean(df.groupby("machine_correct"))
        std_df = df.groupby("machine_correct").std()
    else:
        bin_labels = pd.qcut(df[key], q=bins, labels=np.arange(bins))
        df = df.copy()
        df["bin"] = bin_labels
        mean_df = df.groupby("bin").mean()
        std_df = df.groupby("bin").std()
        mean_df = mean_df.set_index(key)
        std_df.index = mean_df.index

    ax_was_none = False
    if ax is None:
        ax_was_none = True
        _, ax = plt.subplots(1, 1)
    if compact_label:
        label = (f"$\\rho_s = {scipy.stats.spearmanr(mean_df.index, mean_df['human_correct'])[0]:.2f}$\n"
                f"$\\rho_p = {scipy.stats.pearsonr(mean_df.index, mean_df['human_correct'])[0]:.2f}$")
    else:
        label = (f"Average defined by MIS\n$\\rho_s = {scipy.stats.spearmanr(mean_df.index, mean_df['human_correct'])[0]:.2f}, "
              f"\\rho_p = {scipy.stats.pearsonr(mean_df.index, mean_df['human_correct'])[0]:.2f}$")
    ax.plot(mean_df.index, mean_df["human_correct"],
            label=label, c="C0", clip_on=False)
    ax.fill_between(
        mean_df.index,
        mean_df["human_correct"] - std_df["human_correct"],
        mean_df["human_correct"] + std_df["human_correct"],
        alpha=0.5, color="C0"
    )
    ax.scatter(mean_df.index, mean_df["human_correct"], marker="x")
    min_x = mean_df.index.min()
    max_x = mean_df.index.max()
    ax.plot([min_x, max_x], [min_x, max_x], label="Oracle MIS=HIS", c="C1", ls=":")
    ax.set_xlabel("Machine Interpretability Score (MIS)")
    ax.set_ylabel("Human Interpretability Score (HIS)")
    ax.legend(frameon=False)
    ax.spines[["top", "right"]].set_visible(False)
    if ax_was_none:
        plt.show()

In [ ]:
def get_train_test_data(
        main_df: pd.DataFrame,
        mode: str = "natural",
        train_test_split: Union[Literal["model"], Literal["global"]] = "model",
        model_name: str = "googlenet",
        train_test_ratio: float = 1.0,
        shuffle_features: bool = False,
        similarity_key: str = "dreamsim_similarity"
    ) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:

    main_subset_df = main_df[main_df["mode"] == mode].copy()

    main_subset_df = main_subset_df.rename(columns={"correct": "human_correct"})

    def add_features_and_labels(df: pd.DataFrame):
        # Duplicate every row so that we can add the flipped images/labels.
        df = df.loc[df.index.repeat(2)].reset_index(drop=True)

        labels = []
        features = []

        if len(df) > 0:
            raw_features = np.stack(df[similarity_key].apply(lambda x: (x[:9, 9], x[:9, -1], x[10:-1, 9], x[10:-1, -1])))
    
            for i in range(0, len(raw_features), 2):
                if shuffle_features:
                    idx_1 = np.random.permutation(range(0, 8)).tolist() + [8]
                    idx_2 = np.random.permutation(range(0, 8)).tolist() + [8]
                else:
                    idx_1 = idx_2 = list(range(raw_features.shape[-1]))

                features.append(np.concatenate((raw_features[i][0][idx_1], raw_features[i][1][idx_1], raw_features[i][2][idx_2], raw_features[i][3][idx_2])))
                features.append(np.concatenate((raw_features[i][1][idx_1], raw_features[i][0][idx_1], raw_features[i][3][idx_2], raw_features[i][2][idx_2])))
                labels.append(1)
                labels.append(0)
                
        features = np.array(features)
        labels = np.array(labels)

        df["features"] = pd.Series(features.tolist(), dtype=np.dtype("object"))
        df["label"] = pd.Series(labels)

        df["features"] = df["features"].apply(lambda x: np.array(x))
        
        return df

    df_train_mask = None
    df_test_mask = None

    if train_test_split == "model":
        df_train_mask = main_subset_df["model"] == model_name
        n = int(train_test_ratio * sum(df_train_mask))
        for i in range(len(df_train_mask)):
            if df_train_mask.iloc[i]:
                n -= 1
            if n < 0:
                df_train_mask.iloc[i:] = False
                break
    else:
        shuffled_idxs = np.random.permutation(len(main_subset_df))
        df_train_mask = np.zeros(len(main_subset_df), dtype=bool)
        n_train = int(train_test_ratio * len(main_subset_df))
        train_idxs = shuffled_idxs[:n_train]
        df_train_mask[train_idxs] = True

    df_test_mask = ~df_train_mask

    train_df = main_subset_df[df_train_mask]
    test_df = main_subset_df[df_test_mask]

    train_df = add_features_and_labels(train_df)
    test_df = add_features_and_labels(test_df)

    full_df = pd.concat((train_df, test_df), ignore_index=True)

    return train_df, test_df, full_df

In [ ]:
def fit_eval_classification_model(model, x_train, y_train, x_test, y_test, verbose: bool = True):
    preprocessing_x = sklearn.preprocessing.StandardScaler()
    x_train_tf = preprocessing_x.fit_transform(x_train)
    
    #class DummyProcessor:
    #    def transform(self, x):
    #        return x
    #preprocessing_x = DummyProcessor()
    #x_train_tf = preprocessing_x.transform(x_train)
    x_test_tf = preprocessing_x.transform(x_test)

    model = model.fit(x_train_tf, y_train)

    y_train_pred = model.predict(x_train_tf)
    y_test_pred = model.predict(x_test_tf)

    if y_train_pred.ndim == y_train.ndim - 1:
        y_train_pred = y_train_pred[..., None]
    if y_test_pred.ndim == y_test.ndim - 1:
        y_test_pred = y_test_pred[..., None]

    train_accuracy = (y_train_pred == y_train).mean()
    test_accuracy = (y_test_pred == y_test).mean()
    if verbose:
        print("Train accuracy:", train_accuracy, "Test accuracy:", test_accuracy)

    return (
        (y_train_pred, y_test_pred),
        (train_accuracy, test_accuracy),
        (preprocessing_x, lambda x: preprocessing_x.transform(x))
    )

In [ ]:
def evaluate_model(predict_fn, get_probs_fn, test_df, machine_k: str = "machine_confidence", full_evaluation: bool = True,
                   highlight_model: Optional[str] = None, legend: bool = True, color: str = "C0", errorbar: bool = True, oracle: bool = False, ax=None,
                   only_return_data: bool = False):
    df_test = get_model_interpretability_scores(predict_fn, get_probs_fn, test_df).reset_index()

    confidences = {k: scipy.stats.bootstrap(
        df_test[df_test["model"] == k]["human_correct"].values.reshape(
        (1, -1)), statistic=np.mean, n_resamples=1_0_000) for k in df_test["model"].unique()}

    means_y = {k: df_test[df_test["model"] == k]["human_correct"].mean(numeric_only=True) for k in df_test["model"].unique()}
    means_x = {k: df_test[df_test["model"] == k][machine_k].mean(numeric_only=True) for k in df_test["model"].unique()}

    if only_return_data:
        return means_x, means_y, confidences
    
    if ax is None:
        fig, ax = plt.subplots(1, 1, figsize=(3.1, 3.1))

    confidences = np.array([(means_y[k] - confidences[k].confidence_interval[0], confidences[k].confidence_interval[1] - means_y[k]) for k in sorted(confidences.keys())])
    means_x_np = np.array([means_x[k] for k in sorted(means_x.keys())])
    means_y_np = np.array([means_y[k] for k in sorted(means_y.keys())])

    corr = (scipy.stats.spearmanr(means_x_np, means_y_np)[0], scipy.stats.pearsonr(means_x_np, means_y_np)[0])
    print("Correlations:", corr)

    if highlight_model:
        colors = [color if k != highlight_model else "C1" for k in sorted(means_x.keys())]
    else:
        colors = color

    ax.scatter(means_x_np, means_y_np, label="Models\n$\\rho_s = {:.2f}$\n$\\rho_p = {:.2f}$".format(*corr), c=colors, clip_on=False)
    if errorbar:
        ax.errorbar(means_x_np, means_y_np, yerr=confidences.T, fmt="none", ecolor=colors, alpha=0.5, clip_on=False)
    if oracle:
        ax.plot([np.min(means_y_np), np.max(means_y_np)], [np.min(means_y_np), np.max(means_y_np)], color="k", ls=":", label="Oracle MIS=HIS")

    ax.set_ylabel("Human Interpretability Score")
    ax.set_xlabel("Machine Interpretability Score")
    ax.spines[["top", "right"]].set_visible(False)
    ax.spines["bottom"].set_bounds(0.60, 0.9)
    ax.set_xlim(0.57, 0.95)

    if means_y_np.max() > 0.8:
        ax.set_ylim(0.57, 0.9)
        ax.spines["left"].set_bounds(0.60, 0.9)
    else:
        ax.set_ylim(0.50, 0.8)
        ax.spines["left"].set_bounds(0.55, 0.8)

    if legend:
        ax.legend(frameon=False, loc="upper left", bbox_to_anchor=(-0.05, 1.0), handletextpad=0.1)
    plt.tight_layout()

    if full_evaluation:
        plt.show()
        fig, axs = plt.subplots(1, 2, figsize=(10, 3), sharey=True)
        plot_raw_human_machine_accuracy_data(df_test, machine_k, ax=axs[0])
        plot_binned_human_machine_accuracy_data(df_test, machine_k, ax=axs[1], bins="unique" if machine_k == "machine_correct" else 20)
        plt.show()

In [ ]:
%cd ../../stimuli_generation
from mis_utils import SimpleBinaryClassifier
%cd ../tools/data_analysis

In [ ]:
def test_hard_coded_models(eval_kwargs={}, similarity_key="dreamsim_similarity", mode: str = "natural"):
    train_df, test_df, full_df = get_train_test_data(main_df, mode=mode, model_name=None, train_test_split="global", train_test_ratio=0.05, similarity_key=similarity_key)
    for aggregation_f, aggregation_n in zip((np.mean, np.max, np.min), ("mean",)): #  "min", "max"
        print(f"Using hard-coded classifier with {aggregation_n} aggregation")
        
        model = SimpleBinaryClassifier(aggregation_f)
        #preprocessing_x = sklearn.preprocessing.StandardScaler()
        #preprocessing_x.fit(np.stack(train_df["features"].values, 0))
        #preprocess_fn = lambda x: preprocessing_x.transform(x)

    	# Determined roughly through a grid search (see commented block below) on the train data

        if similarity_key == "dreamsim_similarity":
            # tuned for dreamsim
            var = 0.16
        elif similarity_key == "lpips_similarity":
            # tuned for lpips
            var = 0.08 # 1.0
        elif similarity_key == "dists_similarity":
            # tuned for dists
            var = 0.08

        preprocessing_x  = sklearn.preprocessing.StandardScaler()
        preprocessing_x.scale_ = np.array([var] * 36)
        preprocess_fn = preprocessing_x.transform
        preprocessing_x.mean_ = np.array([0.0] * 36)

        """
        model = sklearn.linear_model.LogisticRegression()
        model.coef_ = np.concatenate(([1/9] * 9, [-1/9] * 9, [-1/9] * 9, [1/9] * 9)).reshape(1, -1)
        model.intercept_ = np.array([0.0])
        model.classes_ = np.array([0, 1])
        """
        
        predict_fn = lambda x: model.predict(preprocess_fn(x))
        get_probs_fn = lambda x, y: model.predict_proba(preprocess_fn(x))[range(len(y)), y]
           
        """
        var_min = 0.01
        var_max = 0.25
        # Perform grid search to find the best value for var
        n_steps = 100
        res = {}
        for i in range(n_steps):
            var = var_min + (var_max - var_min) / n_steps * i
            preprocessing_x.scale_ = np.array([var] * 36)
            df_test = get_model_interpretability_scores(predict_fn, get_probs_fn, train_df).reset_index()
            means_y = {k: df_test[df_test["model"] == k]["human_correct"].mean(numeric_only=True) for k in df_test["model"].unique()}
            means_x = {k: df_test[df_test["model"] == k]["machine_confidence"].mean(numeric_only=True) for k in df_test["model"].unique()}
            means_x_np = np.array([means_x[k] for k in sorted(means_x.keys())])
            means_y_np = np.array([means_y[k] for k in sorted(means_y.keys())])
            correlation = scipy.stats.pearsonr(means_y_np, means_x_np)[0]
            mse = ((means_y_np - means_x_np) ** 2).mean()
            res[var] = (mse, correlation)

        return res
        """

        return evaluate_model(predict_fn, get_probs_fn, full_df, full_evaluation=False, **eval_kwargs)

def test_learned_models():
    for model_name in main_df["model"].unique():
        print(f"Training LR on Data for Model: {model_name}")
        train_df, test_df, full_df = get_train_test_data(main_df, model_name=model_name, train_test_split="model")
        model = sklearn.linear_model.LogisticRegression(fit_intercept=False, max_iter=10_000, tol=1e-10)
        (
            (_, _),
            (train_accuracy, test_accuracy),
            (preprocessing_x, preprocess_fn)
        ) = fit_eval_classification_model(
            model,
            np.stack(train_df["features"].values, 0),
            train_df["label"],
            np.stack(test_df["features"].values, 0),
            test_df["label"]
        )

        predict_fn = lambda x: model.predict(preprocess_fn(x))
        get_probs_fn = lambda x, y: model.predict_proba(preprocess_fn(x))[range(len(y)), y]
        evaluate_model(predict_fn, get_probs_fn, full_df, full_evaluation=False, highlight_model=model_name)
        plt.savefig(f"{model_name}.pdf", bbox_inches="tight")
        plt.show()

In [ ]:
test_hard_coded_models(eval_kwargs={"legend": False, "color": "k", "errorbar": False, "oracle": True})
plt.gca().set_xticklabels([""] * 4)
plt.gca().set_yticklabels([""] * 7)
plt.tight_layout()
plt.savefig(f"results_metrics/model_correlation_dreamsim_logistic_regression_hard_coded_figure_1.pdf", bbox_inches="tight")
plt.show()
#test_learned_models()

In [ ]:
test_hard_coded_models(mode="optimized")
plt.gca().set_xlim(0.53, 0.70)
plt.gca().set_ylim(0.53, 0.75)
plt.gca().spines["bottom"].set_bounds(0.55, 0.70)
plt.gca().spines["left"].set_bounds(0.55, 0.75)
plt.tight_layout()
plt.savefig(f"results_metrics/model_correlation_optimized_dreamsim_logistic_regression_hard_coded.pdf", bbox_inches="tight")

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(5, 2.9), sharey=True)
for ax, k in zip(axs, ("lpips_similarity", "dists_similarity")):
    test_hard_coded_models(similarity_key=k, eval_kwargs={"ax": ax})
axs[1].set_ylabel("")
axs[0].set_xlabel("MIS (LPIPS)")
axs[1].set_xlabel("MIS (DISTS)")
plt.tight_layout()
plt.savefig(f"results_metrics/model_correlation_lpips_dists_logistic_regression_hard_coded.pdf", bbox_inches="tight")

In [ ]:
for k in ("dreamsim_similarity", "lpips_similarity", "dists_similarity"):
    test_hard_coded_models(similarity_key=k)
    plt.tight_layout()
    plt.savefig(f"results_metrics/model_correlation_{k.split('_')[0]}_logistic_regression_hard_coded.pdf", bbox_inches="tight")
    plt.show()

In [ ]:
imi_print_name_map = {
    'clip-resnet50': "Clip ResNet-50",
    'clip-resnet50_hard85': "Clip ResNet-50 (Very Hard)",
    'clip-resnet50_hard95': "Clip ResNet-50 (Hard)",
    'clip-resnet50_hard99': "Clip ResNet-50 (Medium)",
    'clip-vit_b32': "Clip ViT-B32",
    'convnext_b': "ConvNeXT-B",
    'densenet_201': "DenseNet-201",
    'googlenet': "GoogLeNet",
    'in1k-vit_b32': "ViT-B32",
    'resnet50': "ResNet-50",
    'resnet50-l2': "Robust ResNet-50",
    'resnet50_hard85': "ResNet-50 (Very Hard)",
    'resnet50_hard95': "ResNet-50 (Hard)",
    'resnet50_hard99': "ResNet-50 (Medium)",
    'wide_resnet50': "WideResNet-50",
}

In [ ]:
def show_units_with_worst_predictions(similarity_key: str = "dreamsim_similarity", exclude_hard: bool = True):
    model = SimpleBinaryClassifier(np.mean)

    if similarity_key == "dreamsim_similarity":
        # tuned for dreamsim
        var = 0.16
    elif similarity_key == "lpips_similarity":
        # tuned for lpips
        var = 0.08
    elif similarity_key == "dists_similarity":
        # tuned for dists
        var = 0.10

    preprocessing_x  = sklearn.preprocessing.StandardScaler()
    preprocessing_x.scale_ = np.array([var] * 36)
    preprocess_fn = preprocessing_x.transform
    preprocessing_x.mean_ = np.array([0.0] * 36)

    predict_fn = lambda x: model.predict(preprocess_fn(x))
    get_probs_fn = lambda x, y: model.predict_proba(preprocess_fn(x))[range(len(y)), y]

    _, _, full_df = get_train_test_data(main_df, model_name=None, train_test_split="global", train_test_ratio=0.05, similarity_key=similarity_key)
    df_test = get_model_interpretability_scores(predict_fn, get_probs_fn, full_df).reset_index()

    if exclude_hard:
        df_test = df_test[~df_test["model"].str.contains("hard")]

    df_test = df_test.set_index(["model", "channel", "layer"])
    diff = df_test["machine_confidence"] - df_test["human_correct"]
    diff = diff.sort_values()
    display(diff.iloc[:10])
    display(diff.iloc[-10:])

In [ ]:
show_units_with_worst_predictions(similarity_key="dreamsim_similarity")

In [ ]:
def get_test_data(similarity_key: str = "dreamsim_similarity", mode: str = "natural"):
    model = SimpleBinaryClassifier(np.mean)
    if similarity_key == "dreamsim_similarity":
        # tuned for dreamsim
        var = 0.16
    elif similarity_key == "lpips_similarity":
        # tuned for lpips
        var = 0.08
    elif similarity_key == "dists_similarity":
        # tuned for dists
        var = 0.10
    
    preprocessing_x  = sklearn.preprocessing.StandardScaler()
    preprocessing_x.scale_ = np.array([var] * 36)
    preprocess_fn = preprocessing_x.transform
    preprocessing_x.mean_ = np.array([0.0] * 36)

    predict_fn = lambda x: model.predict(preprocess_fn(x))
    get_probs_fn = lambda x, y: model.predict_proba(preprocess_fn(x))[range(len(y)), y]

    _, _, full_df = get_train_test_data(main_df, mode=mode, model_name=None, train_test_split="global", train_test_ratio=0.05, similarity_key=similarity_key)
    df_test = get_model_interpretability_scores(predict_fn, get_probs_fn, full_df).reset_index()

    return df_test

## Compare Similarities

In [ ]:
def plot_unit_correlation(similarity_key: str = "dreamsim_similarity", axs=None, binned_kwargs={}, raw_kwargs={}, hide_xlabel: bool = False):
    df_test = get_test_data(similarity_key)

    if axs is None:
        fig, axs = plt.subplots(1, 2, figsize=(6, 3), sharey=True)
    plot_raw_human_machine_accuracy_data(df_test, "machine_confidence", ax=axs[0], **raw_kwargs)
    plot_binned_human_machine_accuracy_data(df_test, "machine_confidence", ax=axs[1], bins=20, **binned_kwargs)

    axs[0].set_xlabel("")
    axs[1].set_xlabel("")
    if not hide_xlabel:
        axs[0].get_figure().text(0.5, 0.00, "Machine Interpretability Score (MIS)", ha='center')
    axs[1].set_ylabel("")
    axs[0].set_ylim(0.28, 1.0)
    axs[0].spines["left"].set_bounds(0.3, 1.0)
    axs[0].set_xlim(0.385, 1.0)
    axs[0].spines["bottom"].set_bounds(0.4, 1.0)
    axs[1].set_xlim(0.485, 1.0)
    axs[1].spines["bottom"].set_bounds(0.5, 1.0)
    plt.tight_layout()

In [ ]:
fig, axs = plt.subplots(1, 6, figsize=(14, 2.9), sharey=True)
plot_unit_correlation("dreamsim_similarity", axs=axs[[0, 1]], binned_kwargs={"compact_label": True}, hide_xlabel=True)
plot_unit_correlation("lpips_similarity", axs=axs[[2, 3]], binned_kwargs={"compact_label": True}, hide_xlabel=True)
plot_unit_correlation("dists_similarity", axs=axs[[4, 5]], binned_kwargs={"compact_label": True}, hide_xlabel=True)
axs[2].set_ylabel("")
axs[4].set_ylabel("")
plt.tight_layout()
fig.text(0.205, 0.00, "MIS (DreamSim)", ha='center')
fig.text(0.525, 0.00, "MIS (LPIPS)", ha='center')
fig.text(0.845, 0.00, "MIS (DISTS)", ha='center')
plt.savefig(f"results_metrics/unit_correlation_dreamsim_lpips_dists_logistic_regression_hard_coded.pdf", bbox_inches="tight")

In [ ]:
plot_unit_correlation("dreamsim_similarity")
plt.savefig(f"results_metrics/unit_correlation_dreamsim_logistic_regression_hard_coded.pdf", bbox_inches="tight")
plot_unit_correlation("lpips_similarity")
plt.savefig(f"results_metrics/unit_correlation_lpips_logistic_regression_hard_coded.pdf", bbox_inches="tight")
plot_unit_correlation("dists_similarity")
plt.savefig(f"results_metrics/unit_correlation_dists_logistic_regression_hard_coded.pdf", bbox_inches="tight")

### Compute Noise Ceiling

In [ ]:
def get_sample(p, num_trial, seed=42):
    num_task = p.shape[0]
    rng = np.random.default_rng(seed=seed)
    p_ext = np.ones((num_task, num_trial)) * p[:, None]
    return scipy.stats.bernoulli.rvs(p_ext, size=(num_task, num_trial), random_state=rng)
    

df_test = get_test_data("dreamsim_similarity")

# Run Simulations
p = df_test.human_correct.to_numpy()
num_rep = 1000
corrs_p = np.zeros(num_rep)
corrs_s = np.zeros(num_rep)
seed = 1302024

for num_trial in [20, 25, 30]:
    for i in range(num_rep):
        X1 = get_sample(p, num_trial, seed=seed + i)
        X2 = get_sample(p, num_trial, seed=seed + num_rep + i)
        corrs_p[i] = scipy.stats.pearsonr(X1.mean(1), X2.mean(1))[0]
        corrs_s[i] = scipy.stats.spearmanr(X1.mean(1), X2.mean(1))[0]

    print('Num Trial:', num_trial)
    print('Linear Correlation: N=%s, Mean=%s, Std=%s' % (
        num_rep, np.mean(corrs_p), np.std(corrs_p))
    )
    print('Rank Correlation: N=%s, Mean=%s, Std=%s' % (
        num_rep, np.mean(corrs_s), np.std(corrs_s))
    )
            

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(6, 2.7), sharey=False)
axs[0].set_ylabel("Pearson Correlation (HIS, MIS)")
axs[1].set_ylabel("Spearman Correlation (HIS, MIS)")
fig.text(0.425, 0, "Perceptual Similarity")

model_corr_p = (0.5038248025289342, 0.7003236706345828, 0.8001065765425636)
model_corr_s = (0.5202567116223824, 0.7006975875959636, 0.8038836149835622)

axs[0].bar(np.arange(3), model_corr_p,)
axs[1].bar(np.arange(3), model_corr_s)

axs[0].bar_label(axs[0].containers[0], fmt=lambda x: f"{x:.2f}", label_type="center", color="white")
axs[1].bar_label(axs[1].containers[0], fmt=lambda x: f"{x:.2f}", label_type="center", color="white")

axs[0].hlines(np.mean(corrs_p), -0.5, 2.5, linestyle="dashed", color="k", lw=1)
axs[0].fill_between([-0.5, 2.5], np.mean(corrs_p) - np.std(corrs_p), np.mean(corrs_p) + np.std(corrs_p), alpha=0.2, color="k")
axs[1].hlines(np.mean(corrs_s), -0.5, 2.5, linestyle="dashed", color="k", lw=1)
axs[1].fill_between([-0.5, 2.5], np.mean(corrs_s) - np.std(corrs_s), np.mean(corrs_s) + np.std(corrs_s), alpha=0.2, color="k")

axs[0].set_xticks([0, 1, 2], ["LPIPS", "DISTS", "DreamSim"])
axs[1].set_xticks([0, 1, 2], ["LPIPS", "DISTS", "DreamSim"])

axs[0].spines[["top", "right"]].set_visible(False)
axs[1].spines[["top", "right"]].set_visible(False)

axs[0].spines["bottom"].set_bounds(-0.5, 2.5)
axs[1].spines["bottom"].set_bounds(-0.5, 2.5)

axs[0].spines["left"].set_bounds(0.0, 1.0)
axs[1].spines["left"].set_bounds(0.0, 1.0)

axs[0].set_ylim(-0.05, 1.01)
axs[1].set_ylim(-0.05, 1.01)

axs[0].text(-0.25, 0.86, "Noise Ceiling")

plt.tight_layout()
plt.savefig(f"results_metrics/model_correlation_noise_ceiling_dreamsim_lpips_dists_logistic_regression_hard_coded.pdf", bbox_inches="tight")


In [ ]:
np.mean(corrs_p), np.mean(corrs_s)

### Save model

In [ ]:
model = SimpleBinaryClassifier(np.mean)
var = 0.16

preprocessing_x  = sklearn.preprocessing.StandardScaler()
preprocessing_x.scale_ = np.array([var] * 36)
preprocess_fn = preprocessing_x.transform
preprocessing_x.mean_ = np.array([0.0] * 36)

predict_fn = lambda x: model.predict(preprocess_fn(x))
get_probs_fn = lambda x, y: model.predict_proba(preprocess_fn(x))[range(len(y)), y.astype(int)]

df_test = get_model_interpretability_scores(
    predict_fn,
    get_probs_fn,
    get_train_test_data(main_df, model_name=None, train_test_split="global", train_test_ratio=0.00)[2]
).reset_index()
#df_test = get_model_interpretability_scores_v4(model, preprocess_fn, main_subset_df, df_test_mask)

df_test = df_test.reset_index()
df_test["layer_index"] = -1
df_test["relative_layer_index"] = -1

model_names = {
    "clip-resnet50_hard85": "clip-resnet50",
    "clip-resnet50_hard95": "clip-resnet50",
    "clip-resnet50_hard99": "clip-resnet50",
    "resnet50_hard85": "resnet50",
    "resnet50_hard95": "resnet50",
    "resnet50_hard99": "resnet50"
}

for model_name in tqdm(df_test["model"].unique()):
    inner_model_name = model_names.get(model_name, model_name)
    inner_model = utils_stimuli_generation.load_model(inner_model_name)
    network_layers = utils_stimuli_generation.get_relevant_layers(inner_model, inner_model_name)

    if model_name == "clip-vit_b32":
        network_layers = ["visual_" + it for it in network_layers]

    df_test.loc[df_test["model"] == model_name, "layer_index"] = df_test.loc[df_test["model"] == model_name, "layer"].map(lambda l: network_layers.index(l))

    df_test.loc[df_test["model"] == model_name, "relative_layer_index"] = df_test.loc[df_test["model"] == model_name, "layer_index"] / df_test.loc[df_test["model"] == model_name, "layer_index"].max()

In [ ]:
model = mis_utils.SimpleBinaryClassifier(np.mean)
preprocessing_x  = sklearn.preprocessing.StandardScaler()
preprocessing_x.scale_ = np.array([0.16] * 36)
preprocessing_x.mean_ = np.array([0.0] * 36)

preprocess_fn = preprocessing_x.transform
_, _, full_df = get_train_test_data(main_df, model_name=None, train_test_split="global", train_test_ratio=0.05, similarity_key="dreamsim_similarity")
predict_fn = lambda x: model.predict(preprocess_fn(x))
get_probs_fn = lambda x, y: model.predict_proba(preprocess_fn(x))[range(len(y)), y]
test_df = get_model_interpretability_scores(predict_fn, get_probs_fn, full_df)

In [ ]:
test_df = test_df[["human_correct", "machine_confidence"]]
test_df = test_df.rename(columns={"human_correct": "HIS", "machine_confidence": "MIS"})

def simulate_binomial(n, p):
    return np.random.binomial(n, p) / n

test_df["HIS_simulated"] = test_df["MIS"].apply(lambda x: simulate_binomial(30, x))
test_df["MIS_simulated"] = test_df["HIS"].apply(lambda x: simulate_binomial(20, x))

In [ ]:
plt.scatter(test_df["MIS"], test_df["HIS"], s=1, c="C0", label="Real Data", clip_on=False)
plt.scatter(test_df["MIS"], test_df["HIS_simulated"], s=1, c="C1", label="Simulate HIS with\nBinomial(p=MIS, n=30)", clip_on=False)
plt.scatter(test_df["MIS_simulated"], test_df["HIS"], s=1, c="C3", label="Simulate MIS with\nBinomial(p=HIS, n=20)", clip_on=False)
plt.legend()
plt.xlabel("MIS")
plt.ylabel("HIS")
plt.xlim(0.25, 1.0)
plt.ylim(0.25, 1.0)
plt.gca().spines[["top", "right"]].set_visible(False)